In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import random
import math
import xgboost as xgb

from copy import copy
import pickle

In [3]:
#load test data
taxi_test = pd.read_csv('data/test.csv')

In [4]:
#load xgb model
model = pickle.load(open('xgb_model.sav', 'rb'))

[20:36:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


# Testing Data
This is for testing the auxiliary functions made. However, functions are in a .py file that will be loaded in our final Genetic Algorithm.

In [5]:
test_set = taxi_test.head(10)

In [6]:
test_set

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N
5,id0668992,1,2016-06-30 23:59:30,1,-73.991302,40.749798,-73.980515,40.786549,N
6,id1765014,1,2016-06-30 23:59:15,1,-73.978310,40.741550,-73.952072,40.717003,N
7,id0898117,1,2016-06-30 23:59:09,2,-74.012711,40.701527,-73.986481,40.719509,N
8,id3905224,2,2016-06-30 23:58:55,2,-73.992332,40.730511,-73.875618,40.875214,N
9,id1543102,2,2016-06-30 23:58:46,1,-73.993179,40.748760,-73.979309,40.761311,N


In [7]:
def clean_df(input_df):
    
    df = input_df
    
    col_list = ['pickup_longitude', 
        'pickup_latitude', 
        'dropoff_longitude', 
        'dropoff_latitude', 
        'trip_duration',
        'pickup_minute',
        'pickup_hour',
        'pickup_month',
        'pickup_day',
        'pickup_weekday']
    
    #changing datetime column to datetime class
    df['pickup_datetime_hold'] = pd.to_datetime(df['pickup_datetime'])
    
    #adding minute column
    df['pickup_minute'] = df.apply(lambda x: x.pickup_datetime_hold.minute, axis = 1) 

    #adding hour column
    df['pickup_hour'] = df.apply(lambda x: x.pickup_datetime_hold.hour, axis = 1) 

    #adding month column
    df['pickup_month'] = df.apply(lambda x: x.pickup_datetime_hold.month, axis = 1) 

    #adding day of month column
    df['pickup_day'] = df.apply(lambda x: x.pickup_datetime_hold.day, axis = 1) 

    #adding day of week column
    df['pickup_weekday'] = df.apply(lambda x: datetime.weekday(x.pickup_datetime_hold), axis = 1) 
    
    return(df[df.columns.intersection(col_list)])

In [8]:
hold = clean_df(test_set)

C:\Users\eldon\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\eldon\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\eldon\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [9]:
hold

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_minute,pickup_hour,pickup_month,pickup_day,pickup_weekday
0,-73.988129,40.732029,-73.990173,40.756680,59,23,6,30,3
1,-73.964203,40.679993,-73.959808,40.655403,59,23,6,30,3
2,-73.997437,40.737583,-73.986160,40.729523,59,23,6,30,3
3,-73.956070,40.771900,-73.986427,40.730469,59,23,6,30,3
4,-73.970215,40.761475,-73.961510,40.755890,59,23,6,30,3
5,-73.991302,40.749798,-73.980515,40.786549,59,23,6,30,3
6,-73.978310,40.741550,-73.952072,40.717003,59,23,6,30,3
7,-74.012711,40.701527,-73.986481,40.719509,59,23,6,30,3
8,-73.992332,40.730511,-73.875618,40.875214,58,23,6,30,3
9,-73.993179,40.748760,-73.979309,40.761311,58,23,6,30,3


In [10]:
#test set
temp =hold[['pickup_longitude', 'pickup_latitude']][0:10]

In [11]:
temp

,pickup_longitude,pickup_latitude
0,-73.988129,40.732029
1,-73.964203,40.679993
2,-73.997437,40.737583
3,-73.956070,40.771900
4,-73.970215,40.761475
5,-73.991302,40.749798
6,-73.978310,40.741550
7,-74.012711,40.701527
8,-73.992332,40.730511
9,-73.993179,40.748760


In [12]:
#Sample date

x = datetime(2020, 5, 17, 17)

print(x)

2020-05-17 17:00:00


# Auxiliary Functions

In [13]:
#converting seconds into hour, minutes, seconds, microseconds
def convert(start_time, add): 
    add = add % (24 * 3600) 
    hour = add // 3600
    add %= 3600
    minutes = add // 60
    add %= 60
      
    return(start_time + timedelta(seconds=add, minutes=minutes, hours=hour)) 

In [14]:
#setup function to only keep columns we want
def setup(x):
    hold = pd.concat([x, x.shift(-1)], axis = 1).dropna()
    hold.columns = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'] 
    return(hold)

In [15]:
#prepare dataframe to be inputted into model
def set_model_input(x, ind, time):
    hold = x.iloc[ind].to_frame().T
    hold['pickup_datetime'] = time
    return(clean_df(hold))

In [16]:
#return total time of a set of trips
def get_total_time(x, start_time):
    
    #setting up intial input df, time, and holding array
    temp = setup(x)
    start_time_hold = start_time
    time_total = [0] * len(temp)
    
    for i in range(len(temp)):
        hold = set_model_input(temp, i, start_time_hold)
        trip_dur = (np.exp(model.predict(xgb.DMatrix(hold))) - 1)[0]
        time_total[i] = trip_dur
        start_time_hold = convert(start_time_hold, trip_dur)
        
    return(sum(time_total))

In [38]:
get_total_time(temp,x)

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.988129        40.732029         -73.964203         40.679993   

   pickup_minute  pickup_hour  pickup_month  pickup_day  pickup_weekday  
0              0           17             5          17               6  
1571.7897
   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
1        -73.964203        40.679993         -73.997437         40.737583   

   pickup_minute  pickup_hour  pickup_month  pickup_day  pickup_weekday  
1             26           17             5          17               6  
2043.464
   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
2        -73.997437        40.737583          -73.95607           40.7719   

   pickup_minute  pickup_hour  pickup_month  pickup_day  pickup_weekday  
2              0           18             5          17               6  
1255.49
   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff

9203.743469238281

# GA functions

In [17]:
#get intial population for GA
def get_init_pop(x, num):
    
    hold = []
    pop = list(range(x))
    
    for i in range(num):
        hold.append((random.sample(pop,x)))
        
    return(hold)

In [25]:
init_pop =  get_init_pop(len(temp),100)

In [26]:
init_pop

[[6, 3, 8, 2, 4, 5, 1, 7, 9, 0],
 [5, 3, 4, 6, 7, 8, 2, 1, 9, 0],
 [6, 5, 1, 3, 4, 9, 7, 8, 2, 0],
 [7, 4, 5, 9, 3, 1, 8, 2, 6, 0],
 [7, 8, 5, 0, 9, 4, 1, 3, 6, 2],
 [3, 2, 6, 4, 7, 0, 9, 1, 8, 5],
 [6, 3, 9, 0, 7, 2, 1, 5, 4, 8],
 [3, 7, 0, 4, 1, 6, 2, 5, 8, 9],
 [3, 6, 0, 5, 9, 8, 4, 1, 2, 7],
 [9, 6, 5, 1, 2, 8, 0, 7, 4, 3],
 [9, 1, 4, 7, 0, 3, 8, 6, 2, 5],
 [7, 2, 5, 6, 4, 0, 8, 9, 1, 3],
 [5, 6, 0, 1, 2, 4, 7, 9, 8, 3],
 [8, 6, 0, 7, 2, 3, 9, 5, 1, 4],
 [1, 9, 5, 6, 3, 8, 0, 7, 4, 2],
 [8, 5, 2, 7, 1, 6, 3, 4, 9, 0],
 [8, 0, 3, 6, 1, 9, 4, 5, 2, 7],
 [9, 8, 1, 7, 6, 2, 4, 3, 5, 0],
 [2, 4, 5, 0, 3, 7, 8, 9, 1, 6],
 [2, 0, 3, 1, 7, 6, 8, 9, 4, 5],
 [0, 6, 8, 2, 4, 1, 7, 9, 5, 3],
 [5, 1, 7, 4, 0, 9, 6, 2, 3, 8],
 [2, 1, 3, 0, 5, 7, 8, 4, 6, 9],
 [0, 3, 4, 5, 9, 1, 7, 8, 2, 6],
 [2, 1, 9, 7, 3, 0, 4, 8, 5, 6],
 [0, 3, 4, 7, 2, 9, 1, 6, 8, 5],
 [4, 8, 1, 0, 5, 9, 6, 3, 2, 7],
 [8, 6, 4, 2, 3, 9, 1, 5, 0, 7],
 [1, 3, 2, 8, 5, 9, 0, 7, 4, 6],
 [2, 9, 7, 5, 6, 4, 3, 0, 8, 1],
 [5, 9, 7,

In [20]:
#this is a supplmentary function which will help with our threshold
def condense (input_df):
    return(input_df.groupby(list(set(input_df.columns) - set(['rank', 'chance']))).sum().reset_index())

In [21]:
#TEST
def rank(input_pop, orig_pts, datetime):
    
    hold_df = pd.DataFrame(input_pop)
    condensed = hold_df.drop_duplicates()
    
    total_time_hold = [0] * len(condensed)
    
    for i in range(len(condensed)):
        total_time_hold[i] = get_total_time(orig_pts.reindex(input_pop[i]), datetime)
    
    #add total time column / fitness 
    condensed['total_time'] = total_time_hold
    
    joined_df = pd.merge(hold_df, condensed,how='left', on=list(hold_df.columns))

    #add rank column which gives the highest rank to the combination with the lowest total trip duration
    joined_df['rank'] = len(joined_df) - joined_df['total_time'].rank() + 1
    
    #add chance column which uses rank to calculate probability of being chosen as a parent
    ##higher fitness means higher chance of breeding
    joined_df['chance'] = joined_df['rank']*2/(len(joined_df)*(len(joined_df)+1))
    
    return(joined_df)   

In [71]:
#prepare/rank intial population
def rank(input_pop, orig_pts, datetime):
    
    hold_df = pd.DataFrame(input_pop)
    
    total_time_hold = [0] * len(input_pop)
    
    for i in range(len(input_pop)):
        total_time_hold[i] = get_total_time(orig_pts.reindex(input_pop[i]), datetime)
    
    #add total time column / fitness 
    hold_df['total_time'] = total_time_hold
    
    #add rank column which gives the highest rank to the combination with the lowest total trip duration
    hold_df['rank'] = len(hold_df) - hold_df['total_time'].rank() + 1
    
    #add chance column which uses rank to calculate probability of being chosen as a parent
    ##higher fitness means higher chance of breeding
    hold_df['chance'] = hold_df['rank']*2/(len(hold_df)*(len(hold_df)+1))
    
    return(hold_df)   

In [72]:
rank(init_pop, temp, x)

,0,1,2,3,4,5,6,7,8,9,total_time,rank,chance
0,8,3,9,7,1,2,0,5,6,4,9255.202148,3.0,0.107143
1,3,8,4,7,9,2,1,0,5,6,10049.192932,1.0,0.035714
2,2,4,9,8,0,7,1,3,5,6,8049.039124,7.0,0.250000
3,2,7,6,1,4,0,8,9,5,3,8349.838654,6.0,0.214286
4,9,4,1,2,7,3,0,6,8,5,9761.288147,2.0,0.071429
5,1,8,6,5,7,3,2,0,9,4,8985.310486,4.0,0.142857
6,0,3,4,2,8,6,5,7,1,9,8680.374786,5.0,0.178571


This portion was to test whether if 2 lines have the same total time what happens with rank and chance. Does not affect the choosing of parents but want to group for possible threshold stopping criteria

In [105]:
init_pop

[[1, 0, 2, 3, 4], [0, 2, 3, 1, 4], [3, 4, 0, 1, 2], [3, 4, 0, 1, 2]]

Create function to get a set of parents from the new input data frame

In [44]:
#choose a set parents from our population based on the fitness
##parents shouls be equal 
def get_parent(input_df):
    pop_size = len(input_df)
    which_parent = np.random.choice(pop_size, pop_size, p = input_df['chance'])
    num_chrom = len(list(set(input_df.columns) - set(['total_time', 'rank', 'chance'])))
    parent = input_df.reindex(which_parent).iloc[:,:num_chrom].to_numpy()
    return(parent)

We want half of chromosomes from each parent. If there are duplicates we will randomly choose with the options left.

In [45]:
#cross parent chromosomes
def cross(parent1, parent2):
    
    #how many chromosomes we want to keep from each parent
    num_parent = len(parent1)
    num_chrom = len(parent1[0])
    num_one = math.ceil(num_chrom/2)
    num_zero = num_chrom - math.ceil(num_chrom/2) #safer option than using floor
    
    chrom_filter_source = ([1] * num_one) + ([0]*num_zero)
    
    #create #create a "filter" for our chromosomes
    chrom_filter = []
    for i in range(num_parent):
        chrom_filter.append((random.sample(chrom_filter_source, num_chrom)))
        
    #create opposite filter for our second parent
    chrom_filter2 = abs(np.subtract(chrom_filter, 1))
    
    #need to add 1 because our lowest number right now is 0 if left alone when filtered, we will have multipl 0's
    new_parent1 = np.multiply(np.add(parent1,1), chrom_filter)  
    new_parent2 = np.multiply(np.add(parent2,1), chrom_filter2)
    
    #loop through all parents and cross chromosomes
    for i in range(num_parent):
        #only look at couples with matching chromosomes
        if(any(x in new_parent1[i] for x in new_parent2[i])):
            #available chromosomes to choose from to fill gaps
            not_set = list(set(list(range(num_chrom+1))[1:]) - set(new_parent1[i] + new_parent2[i]))
            #fill gaps
            for idx, j in enumerate(new_parent2[i]):
                if (j in new_parent1[i] and j != 0):
                    insert = random.sample(not_set, 1)[0]
                    new_parent1[i,idx] = insert
                    new_parent2[i,idx] = 0
                    not_set.remove(insert) #once a chromosome is used remove it from possible choices
    
    #add them together as a cross
    return(np.add(new_parent1, new_parent2))

In [55]:
#getting the next generation
def get_next_gen(input_df):
    p1 = get_parent(input_df)
    p2 = get_parent(input_df)
    next_gen = np.subtract(cross(p1, p2),1)
    return(next_gen)
    

In [56]:
get_next_gen(rank(init_pop, temp, x))

array([[0, 3, 4, 2, 8, 6, 5, 7, 1, 9],
       [0, 5, 9, 2, 8, 7, 1, 3, 4, 6],
       [5, 7, 6, 2, 4, 0, 8, 9, 1, 3],
       [7, 3, 4, 2, 0, 6, 5, 1, 8, 9],
       [0, 3, 4, 2, 8, 6, 1, 7, 5, 9],
       [2, 4, 6, 1, 8, 9, 0, 7, 5, 3],
       [8, 3, 9, 7, 6, 5, 2, 0, 1, 4]], dtype=int64)